In [1]:
print("Hello World")

Hello World


In [2]:
%pip install openai 
%pip install python-dotenv

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.1/85.1 kB 1.7 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 262.9/262.9 kB 5.1 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.8/77.8 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 394.9/394.9 kB 6.4 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 13.0 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.1 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [4]:
import os
from openai import OpenAI

Save the OPENAI_KEY into ".env" file, and *** AND .env TO YOUR .gitignore FILE. ***

In [6]:
from dotenv import load_dotenv

load_dotenv()  # take environment variables from .env.

True

For normal use

In [15]:
MODEL = "gpt-4"
# MODEL = "gpt-3.5-turbo"

client = OpenAI(
    # This is the default and can be omitted
    api_key=os.environ.get("OPENAI_API_KEY"),
)

chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": "Who are you?",
        }
    ],
    model=MODEL,
)

# verify the output
generated_text = chat_completion.choices[0].message.content
print(generated_text)

I am OpenAI's language model, known as GPT-3. I am not a person, but AI software developed to understand and generate human-like text based on the text that I am given.


Verify the ***gpt-3.5-turbo***

In [12]:
client = OpenAI()

completion = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    # {"role": "system", "content": "You are a poetic assistant, skilled in explaining complex programming concepts with creative flair."},
    # {"role": "user", "content": "Compose a poem that explains the concept of recursion in programming."}
    {"role": "user", "content": "Are you GPT4 or GPT3.5?"}
  ]
)

print(completion.choices[0].message.content)

I am an instance of OpenAI's GPT-3 model.
